### Generate the big co-occurrence matrix

1. load the dictionary and add numeric indices
2. Load the song data into dataframe
3. Convert the word keys to words
4. (skip) replace word stems with full words
5. For each row, sum the counts in each column
6. This is your co-occurence matrix

In [1]:
import pandas as pd

In [2]:
# reading csv file  
# matched = pd.read_csv(r"../Py code/words_songs_matched.csv") 

In [3]:
matched.head()

,word,count,track_id,song_id,artist_id,title,artist_name,duration,year,artist_hotttnesss
0,i,6,TRAAAAV128F421A322,SOQPWCR12A6D4FB2A3,AR73AIO1187B9AD57B,A Poor Recipe For Civic Cohesion,Western Addiction,118.07302,2005,0.386606
1,the,4,TRAAAAV128F421A322,SOQPWCR12A6D4FB2A3,AR73AIO1187B9AD57B,A Poor Recipe For Civic Cohesion,Western Addiction,118.07302,2005,0.386606
2,you,2,TRAAAAV128F421A322,SOQPWCR12A6D4FB2A3,AR73AIO1187B9AD57B,A Poor Recipe For Civic Cohesion,Western Addiction,118.07302,2005,0.386606
3,to,2,TRAAAAV128F421A322,SOQPWCR12A6D4FB2A3,AR73AIO1187B9AD57B,A Poor Recipe For Civic Cohesion,Western Addiction,118.07302,2005,0.386606
4,and,5,TRAAAAV128F421A322,SOQPWCR12A6D4FB2A3,AR73AIO1187B9AD57B,A Poor Recipe For Civic Cohesion,Western Addiction,118.07302,2005,0.386606


In [4]:
matched.drop(columns=['track_id', 'song_id', 'artist_id', 'duration', 'artist_hotttnesss'], inplace=True)

In [5]:
matched.head()

,word,count,title,artist_name,year
0,i,6,A Poor Recipe For Civic Cohesion,Western Addiction,2005
1,the,4,A Poor Recipe For Civic Cohesion,Western Addiction,2005
2,you,2,A Poor Recipe For Civic Cohesion,Western Addiction,2005
3,to,2,A Poor Recipe For Civic Cohesion,Western Addiction,2005
4,and,5,A Poor Recipe For Civic Cohesion,Western Addiction,2005


In [10]:
# matched.to_csv(r'../Py code/words_songs_matched_lite.zip', index = False)

### Now we have the trimmed table, time to reswizzle the axes

In [7]:
# find the number of unique words, it should be 5000
uniques = matched.word.unique()

In [8]:
len(uniques)

5000

### Stop using the above code and start from here decreases memory usage since only loading 900MB csv instead of 2GB

In [41]:
import numpy as np
import nltk
from nltk import bigrams
import itertools
import pandas as pd
from chord import Chord

In [11]:
# reading csv file  
matched = pd.read_csv(r"../Py code/words_songs_matched_lite.zip") 

In [19]:
matched.head(100)

,word,count,title,artist_name,year
0,i,6,A Poor Recipe For Civic Cohesion,Western Addiction,2005
1,the,4,A Poor Recipe For Civic Cohesion,Western Addiction,2005
2,you,2,A Poor Recipe For Civic Cohesion,Western Addiction,2005
3,to,2,A Poor Recipe For Civic Cohesion,Western Addiction,2005
4,and,5,A Poor Recipe For Civic Cohesion,Western Addiction,2005
...,...,...,...,...,...
95,get,2,Soul Deep,The Box Tops,1969
96,got,3,Soul Deep,The Box Tops,1969
97,never,2,Soul Deep,The Box Tops,1969
98,feel,2,Soul Deep,The Box Tops,1969


In [13]:
matched.shape

(19045332, 5)

In [20]:
uniquewords = matched.word.unique()
print(uniquewords.shape)

(5000,)


In [28]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
sw = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Wx\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
sw

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [36]:
# drop all stopwords
realwords = matched[~matched['word'].isin(sw)]

In [37]:
realwords.shape

(12789780, 5)

In [71]:
realwords.head(50)

,word,count,title,artist_name,year
22,like,2,A Poor Recipe For Civic Cohesion,Western Addiction,2005
23,de,1,A Poor Recipe For Civic Cohesion,Western Addiction,2005
27,got,1,A Poor Recipe For Civic Cohesion,Western Addiction,2005
28,would,1,A Poor Recipe For Civic Cohesion,Western Addiction,2005
31,seem,1,A Poor Recipe For Civic Cohesion,Western Addiction,2005
32,someon,1,A Poor Recipe For Civic Cohesion,Western Addiction,2005
33,understand,1,A Poor Recipe For Civic Cohesion,Western Addiction,2005
34,pass,1,A Poor Recipe For Civic Cohesion,Western Addiction,2005
35,river,1,A Poor Recipe For Civic Cohesion,Western Addiction,2005
36,met,1,A Poor Recipe For Civic Cohesion,Western Addiction,2005


In [40]:
uniquewords = realwords.word.unique()
print(uniquewords.shape)

(4884,)


In [110]:
finallist= []
wordlist = []

uniq_title = realwords.title.unique()
uniq_title = uniq_title[3:6]

for song in uniq_title:
    wordlist = []
    print(song)
    subset = realwords[realwords['title']==song]
    for word in subset['word']:
        wordlist.append(word)
    finallist.append(wordlist)
        
print(finallist)

Something Girls
Burn My Body (Album Version)
Face the Ashes
[['know', 'got', 'feel', 'let', 'would', 'eye', 'world', 'right', 'think', 'wo', 'girl', 'everi', 'someth', 'tonight', 'old', 'boy', 'mine', 'someon', 'star', 'wish', 'smile', 'shine', 'ask', 'white', 'three', 'send', 'treat', 'relax', 'tip', 'pearl', 'curl', 'adam'], ['like', 'take', 'would', 'wo', 'someth', 'stay', 'burn', 'made', 'use', 'someon', 'breath', 'done', 'touch', 'bodi', 'fade', 'sad', 'stone', 'tight', 'space', 'color', 'mess', 'tie', 'paint', 'cast', 'fix', 'bend', 'melt', 'grey', 'ash', 'wire', 'liar', 'stain', 'wet', 'mix', 'numb', 'rust', 'burnt', 'debt', 'lifeless', 'dissolv', 'mold', 'clay', 'print', 'wax', 'ink', 'leak'], ['one', 'got', 'never', 'feel', 'way', 'take', 'would', 'away', 'could', 'world', 'caus', 'die', 'think', 'dream', 'turn', 'walk', 'face', 'word', 'place', 'better', 'someon', 'kill', 'done', 'matter', 'lay', 'red', 'memori', 'tire', 'clear', 'felt', 'secret', 'wave', 'mouth', 'win', 'exi

In [111]:
# Create one list using many lists
data = list(itertools.chain.from_iterable(finallist))
matrix, vocab_index = generate_co_occurrence_matrix(data)


data_matrix = pd.DataFrame(matrix, index=vocab_index, columns=vocab_index)
print(matrix)
print(type(matrix))
print(data_matrix)
mat2 = matrix.tolist()
names = list(data_matrix.index) 
print(type(names))
print(names)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
<class 'numpy.matrix'>
       take  turn  treat  settl  know  world  space  mold  tight  dissolv  \
take    0.0   0.0    0.0    0.0   0.0    0.0    0.0   0.0    0.0      0.0   
turn    0.0   0.0    0.0    0.0   0.0    0.0    0.0   0.0    0.0      0.0   
treat   0.0   0.0    0.0    0.0   0.0    0.0    0.0   0.0    0.0      0.0   
settl   0.0   0.0    0.0    0.0   0.0    0.0    0.0   0.0    0.0      0.0   
know    0.0   0.0    0.0    0.0   0.0    0.0    0.0   0.0    0.0      0.0   
...     ...   ...    ...    ...   ...    ...    ...   ...    ...      ...   
cast    0.0   0.0    0.0    0.0   0.0    0.0    0.0   0.0    0.0      0.0   
clay    0.0   0.0    0.0    0.0   0.0    0.0    0.0   1.0    0.0      0.0   
touch   0.0   0.0    0.0    0.0   0.0    0.0    0.0   0.0    0.0      0.0   
melt    0.0   0.0    0.0    0.0   0.0    0.0    0.0

In [113]:
Chord(mat2, names).to_html("3song.html")

In [42]:
def generate_co_occurrence_matrix(corpus):
    vocab = set(corpus)
    vocab = list(vocab)
    vocab_index = {word: i for i, word in enumerate(vocab)}

    # Create bigrams from all words in corpus
    bi_grams = list(bigrams(corpus))

    # Frequency distribution of bigrams ((word1, word2), num_occurrences)
    bigram_freq = nltk.FreqDist(bi_grams).most_common(len(bi_grams))

    # Initialise co-occurrence matrix
    # co_occurrence_matrix[current][previous]
    co_occurrence_matrix = np.zeros((len(vocab), len(vocab)))

    # Loop through the bigrams taking the current and previous word,
    # and the number of occurrences of the bigram.
    for bigram in bigram_freq:
        current = bigram[0][1]
        previous = bigram[0][0]
        count = bigram[1]
        pos_current = vocab_index[current]
        pos_previous = vocab_index[previous]
        co_occurrence_matrix[pos_current][pos_previous] = count
    co_occurrence_matrix = np.matrix(co_occurrence_matrix)

    # return the matrix and the index
    return co_occurrence_matrix, vocab_index

In [104]:
text_data = [['i', 'need', 'a', 'Dolla', 'dolla'],
             ['a', 'dolla', 'is', 'what', 'i', 'need'],
             ['hey', 'hey',],
             ['and', 'if', 'i', 'share', 'my', 'story', 'with', 'you'],
             ['would', 'you', 'share', 'your', 'dolla', 'with', 'me']]
print(type(text_data))

<class 'list'>


In [105]:
# Create one list using many lists
data = list(itertools.chain.from_iterable(text_data))
matrix, vocab_index = generate_co_occurrence_matrix(data)


data_matrix = pd.DataFrame(matrix, index=vocab_index,
                             columns=vocab_index)
print(matrix)
print(type(matrix))
print(data_matrix)
mat2 = matrix.tolist()
names = list(data_matrix.index) 
print(type(names))
print(names)

[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.

In [106]:
Chord(mat2, names).to_html("demo.html")

So here's a problem:  
The matched table is roughly 900MB and is 19M * 5 = 95M cells  
The liteframe table is 190K rows times 5000 columns = 950M cells >> 9GB  
How is this an improvement??  

And this is without even the word co-occurence